# Train a Model to Trade on the Stock Market

![](https://ei.marketwatch.com/Multimedia/2018/01/16/Photos/ZH/MW-GB819_stocks_20180116134819_ZH.jpg?uuid=d1d56e68-faed-11e7-9db3-9c8e992d421e)

## Efficient Market Hypothesis

Future prices cannot be predicted by analyzing prices from the past. Excess returns cannot be earned in the long run by using investment strategies based on historical share prices or other historical data. Technical analysis techniques will not be able to consistently produce excess returns.

 

## Technical Analysis


In finance, technical analysis is an analysis methodology for forecasting the direction of prices through the study of past market data, primarily price and volume. The efficacy of technical  analysis is disputed by the efficient-market hypothesis which states that stock market prices are essentially unpredictable.

 

## Can a neural net learn Technical Analysis techniques to predict the market and make good trades?

Are we about to get rich?

In [1]:
#from alpha_vantage.techindicators import TechIndicators
#from alpha_vantage.timeseries import TimeSeries
import random
from indicators import *
from transform import *
import pandas_datareader as web
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
import h2o
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.graph_objs import *
from plotly.tools import FigureFactory as FF
import cufflinks as cf
from plots import *
import numpy as np
import operator
import math

# Suppress unwatned warnings
import warnings
warnings.filterwarnings('ignore')
import logging
logging.getLogger("requests").setLevel(logging.WARNING)

# Set some defaults for how pandas displays
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

### Enter These Variables

In [2]:
stockSymbol = "PINC"
market = "NASDAQ"
invest = 1000
window = 15

start_date = pd.datetime(2013,1,1)
split_validation = pd.datetime(2017,1,1)
split_test = pd.datetime(2018,1,1)

### Let's use a machine learning library called H2O

In [3]:
#h2o.cluster().shutdown()

In [4]:
h2o.init(max_mem_size="12G")
h2o.remove_all()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_162"; OpenJDK Runtime Environment (build 1.8.0_162-8u162-b12-0ubuntu0.16.04.2-b12); OpenJDK 64-Bit Server VM (build 25.162-b12, mixed mode)
  Starting server from /home/olmsteadbrett/.local/lib/python2.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpDI6Gc6
  JVM stdout: /tmp/tmpDI6Gc6/h2o_olmsteadbrett_started_from_python.out
  JVM stderr: /tmp/tmpDI6Gc6/h2o_olmsteadbrett_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,07 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.18.0.7
H2O cluster version age:,15 days
H2O cluster name:,H2O_from_python_olmsteadbrett_0xytqy
H2O cluster total nodes:,1
H2O cluster free memory:,10.67 Gb
H2O cluster total cores:,96
H2O cluster allowed cores:,96
H2O cluster status:,"accepting new members, healthy"


### Connect to Morningstar and Download the Historical Data  

In [5]:
df = web.DataReader(stockSymbol, 'morningstar', start=start_date)
df = df.reset_index(drop=False)
df = df.drop('Symbol', axis=1)
df.set_index('Date', inplace=True)
df = df[['Open','High', 'Low', 'Close','Volume']]


### Plot.ly is a Sweet Charting Library

In [6]:
myPlotlyKey = 'conh5EnFad0Z9Lz6mVWr'
py.sign_in(username='bretto777',api_key=myPlotlyKey)

fig = simpleStockChart(df,stockSymbol,split_validation,split_test)
py.iplot(fig, validate=False)

The draw time for this plot will be slow for clients without much RAM.


### Technical Analysis
Add technical analysis indicators used by the pros! Hopefully our model will learn how these indicators can predict tomorrow's price movements.

In [7]:
%%capture
df.reset_index(drop=False, inplace=True, col_level=0)

typical_price(df, high_col = 'High', low_col = 'Low', close_col = 'Close')
money_flow_index(df, vol_col='Volume')
rsi(df, close_col='Close')
ema(df, period=26, column='Close')
ema(df, period=12, column='Close')
macd(df, period_long=26, period_short=12, period_signal=9, column='Close')
on_balance_volume(df, trend_periods=21, close_col='Close', vol_col='Volume')
price_volume_trend(df, trend_periods=21, close_col='Close', vol_col='Volume')
bollinger_bands(df, trend_periods=20, close_col='Close')
williams_r(df, periods=14, high_col='High', low_col='Low', close_col='Close')
ultimate_oscillator(df, period_1=7,period_2=14, period_3=28, high_col='High', low_col='Low', close_col='Close')
df = df.drop(['uo_bp', 'uo_tr', 'uo_avg_1', 'uo_avg_2', 'uo_avg_3'], axis=1)

In [8]:
fig = technicalAnalysisChart(df)
py.iplot(fig, validate=False)

In [9]:
%%capture
windowTransform(df, window)

In [10]:
df['Close Tomorrow Trend'].replace(0,-1,inplace=True)    

trainingDF = df.loc[df['Date'] <= split_validation]
validationDF = df[df['Date'].isin(pd.date_range(split_validation,split_test))]
testDF = df.loc[df['Date'] > split_test]

trainingDF = trainingDF.dropna(axis=0, how='any')
validationDF = validationDF.dropna(axis=0, how='any')
testDF = testDF.fillna(value=0)

trainingDF.set_index('Date', inplace=True)
validationDF.set_index('Date', inplace=True)
testDF.set_index('Date', inplace=True)

In [11]:
train = h2o.H2OFrame(trainingDF, destination_frame="train.hex")
valid = h2o.H2OFrame(validationDF, destination_frame="valid.hex")
test = h2o.H2OFrame(testDF, destination_frame="test.hex")

train[train['Close Tomorrow Trend'] == 0, 'Close Tomorrow Trend'] = -1
train['Close Tomorrow Trend'] = train['Close Tomorrow Trend'].asfactor()
train = train.drop('Close Tomorrow', axis=1)
train = train.drop('Close Tomorrow Change', axis=1)

valid[valid['Close Tomorrow Trend'] == 0, 'Close Tomorrow Trend'] = -1
valid['Close Tomorrow Trend'] = valid['Close Tomorrow Trend'].asfactor()
valid = valid.drop('Close Tomorrow', axis=1)
valid = valid.drop('Close Tomorrow Change', axis=1)

test[test['Close Tomorrow Trend'] == 0, 'Close Tomorrow Trend'] = -1
test['Close Tomorrow Trend'] = test['Close Tomorrow Trend'].asfactor()
test = test.drop('Close Tomorrow', axis=1)
test = test.drop('Close Tomorrow Change', axis=1)

# Set predictor and response variables
y = 'Close Tomorrow Trend'
x = train.columns
x.remove(y)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [12]:
from h2o.estimators.pca import H2OPrincipalComponentAnalysisEstimator as H2OPCA

pca_decomp = H2OPCA(k=84, transform="NONE", pca_method="gram_s_v_d", use_all_factor_levels=True)
pca_decomp.train(x=x, training_frame=train)
trainReduced = pca_decomp.predict(train)
trainReduced = trainReduced.cbind(train['Close Tomorrow Trend'])

pca_decomp.train(x=x, training_frame=valid)
validReduced = pca_decomp.predict(valid)
validReduced = validReduced.cbind(valid['Close Tomorrow Trend'])

pca_decomp.train(x=x, training_frame=test)
testReduced = pca_decomp.predict(test)
testReduced = testReduced.cbind(test['Close Tomorrow Trend'])

pca Model Build progress: |███████████████████████████████████████████████| 100%
pca prediction progress: |████████████████████████████████████████████████| 100%
pca Model Build progress: |███████████████████████████████████████████████| 100%
pca prediction progress: |████████████████████████████████████████████████| 100%
pca Model Build progress: |███████████████████████████████████████████████| 100%
pca prediction progress: |████████████████████████████████████████████████| 100%


### Try AutoML
See what kind of accuracy we can get from AutoML

In [13]:
from h2o.automl import H2OAutoML

# Set predictor and response variables
y = "Close Tomorrow Trend"
x = trainReduced.columns
x.remove(y)

# Run AutoML until 20 models are built
autoModel = H2OAutoML(max_models = 100)
autoModel.train(x = x, y = y,
          training_frame = trainReduced,
          validation_frame = validReduced,
          leaderboard_frame = testReduced)

leaders = autoModel.leaderboard
leaders

AutoML progress: |████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


model_id,auc,logloss
DeepLearning_grid_0_AutoML_20180430_190847_model_15,0.65625,10.9197
DeepLearning_grid_0_AutoML_20180430_190847_model_65,0.636364,13.0896
DeepLearning_grid_0_AutoML_20180430_190847_model_21,0.63125,12.8476
DeepLearning_grid_0_AutoML_20180430_190847_model_31,0.626136,12.4382
DeepLearning_grid_0_AutoML_20180430_190847_model_78,0.624148,11.0308
GLM_grid_0_AutoML_20180430_190847_model_0,0.621591,1.63155
DeepLearning_grid_0_AutoML_20180430_190847_model_40,0.618182,8.9618
DeepLearning_grid_0_AutoML_20180430_190847_model_58,0.612784,13.3127
DeepLearning_grid_0_AutoML_20180430_190847_model_10,0.612784,13.98
DeepLearning_grid_0_AutoML_20180430_190847_model_73,0.61108,13.227


### Train a Deep Neural Net
Grid search for a good model

In [52]:
from h2o.estimators.deeplearning import H2ODeepLearningEstimator
from h2o.grid.grid_search import H2OGridSearch

# Set predictor and response variables
y = "Close Tomorrow Trend"
x = trainReduced.columns
x.remove(y)

hidden_opt = [[50],
              [100],
              [200],
              #[200,200],
              #[25,25,25],
              #[50,75,75,50],
              #[75,100,100,75],
              [1000],
              [500],
              [800]
              ]
l1_opt = [0.001]
l2_opt = [0.001]
rate_opt= [0.01]
activation_opt=["Rectifier","RectifierWithDropout","Tanh", "TanhWithDropout", "Maxout", "MaxoutWithDropout"]
epochs_opt=[2500,1500,2000]

hyper_parameters = {"hidden":hidden_opt, 
                    "l1":l1_opt, 
                    "l2":l2_opt,
                    "rate":rate_opt, 
                    "activation":activation_opt,
                    "epochs":epochs_opt}

search_criteria = {"strategy":"RandomDiscrete", "max_models":5, "seed":123456}



model_grid = H2OGridSearch(H2ODeepLearningEstimator(),
                           hyper_params=hyper_parameters,
                           search_criteria=search_criteria,
                           grid_id='test10')
model_grid.train(x=x, 
                 y=y,                                   
                 fast_mode=True,                   
                 stopping_rounds=10, 
                 stopping_tolerance=0,
                 stopping_metric='AUC',
                 adaptive_rate=True,                
                 training_frame=trainReduced, 
                 validation_frame=validReduced,
                 score_interval=10)

deeplearning Grid Build progress: |███████████████████████████████████████| 100%


In [43]:
#!pwd

/home/olmsteadbrett/stocks


### Take a Look at the Best Model

In [59]:
#model_gridperf1 = model_grid.get_grid(sort_by='recall', decreasing=False)
#bestModel = model_gridperf1[0]  
h2o.save_model(model=bestModel, path="/home/olmsteadbrett/stocks/AutoML")

#bestModel = h2o.get_model(leaders[0,0])


In [53]:
#bestModel = h2o.load_model(path="/home/brett/ticker/h2ogbm.h2o")

H2OResponseError: Server error water.exceptions.H2OIllegalArgumentException:
  Error: Illegal argument: dir of function: importModel: /home/brett/ticker/h2ogbm.h2o
  Request: POST /99/Models.bin/
    data: {u'dir': '/home/brett/ticker/h2ogbm.h2o'}


In [55]:
importances = bestModel.varimp(use_pandas=True)
importances = importances.loc[:,['variable','relative_importance']].groupby('variable').mean()
importances.sort_values(by="relative_importance", ascending=False).iplot(kind='bar', colors='#5AC4F2', theme='white')

In [56]:
cm = bestModel.confusion_matrix(valid=True)
cm = cm.table.as_data_frame()
confusionMatrix = FF.create_table(cm)
confusionMatrix.layout.height=300
confusionMatrix.layout.width=800
confusionMatrix.layout.font.size=17
py.iplot(confusionMatrix)

In [57]:
pBestModel = pd.concat([bestModel.predict(test_data=testReduced).as_data_frame(use_pandas=True).reset_index(drop=True), testDF.reset_index(drop=False)], axis =1)

deeplearning prediction progress: |███████████████████████████████████████| 100%


In [58]:
prices = df.loc[df['Date'] > split_test]
prices = prices.loc[:,['Date','Close','Close Tomorrow','Close Tomorrow Trend']]
profit = pBestModel.loc[:,['predict']]
profit = pd.concat([prices.reset_index(drop=True), profit.reset_index(drop=True)], axis =1)
profit['predict'] = profit['predict'].astype(np.float64)

profit['profit'] = invest

profit['Best Possible'] = profit.apply(lambda row: ((row['profit'] / row['Close'])*row['Close Tomorrow'])-row['profit'] if row['Close Tomorrow Trend'] > 0 else 0, axis=1)

profit['Randomly Invest'] = profit.apply(lambda row: ((row['profit'] / row['Close'])*row['Close Tomorrow'])-row['profit'] if random.randint(1,1000) > random.randint(1,1000) else 0, axis=1)
profit['profit'] = profit.apply(lambda row: ((row['profit'] / row['Close'])*row['Close Tomorrow'])-row['profit'] if row['predict'] > 0 else 0, axis=1)


print "Times " + stockSymbol + " actually went up    : " + str(profit[profit['Close Tomorrow Trend']>0].count()['Close Tomorrow Trend'])
print "Times " + stockSymbol + " predicted to go up  : " + str(profit[profit['predict']>0].count()['predict'])
print "Profit                        : $" + str(profit.profit.sum())
print "Random investment             : $" + str(profit['Randomly Invest'].sum())
print "Best possible outcome         : $" + str(profit['Best Possible'].sum())

profit



Times PINC actually went up    : 44
Times PINC predicted to go up  : 43
Profit                        : $139.37269429107312
Random investment             : $46.89486315863223
Best possible outcome         : $564.4100842965618


,Date,Close,Close Tomorrow,Close Tomorrow Trend,predict,profit,Best Possible,Randomly Invest
0,2018-01-02,29.69,29.43,-1.0,1.0,-8.757157,0.000000,-8.757157
1,2018-01-03,29.43,29.18,-1.0,-1.0,0.000000,0.000000,0.000000
2,2018-01-04,29.18,29.88,1.0,1.0,23.989034,23.989034,23.989034
3,2018-01-05,29.88,29.86,-1.0,1.0,-0.669344,0.000000,0.000000
4,2018-01-08,29.86,30.09,1.0,1.0,7.702612,7.702612,7.702612
5,2018-01-09,30.09,30.60,1.0,-1.0,0.000000,16.949153,0.000000
6,2018-01-10,30.60,31.26,1.0,-1.0,0.000000,21.568627,0.000000
7,2018-01-11,31.26,31.82,1.0,1.0,17.914267,17.914267,0.000000
8,2018-01-12,31.82,31.82,-1.0,-1.0,0.000000,0.000000,0.000000
9,2018-01-15,31.82,32.27,1.0,-1.0,0.000000,14.142049,14.142049


In [19]:
bestModel


Model Details
H2ODeepLearningEstimator :  Deep Learning
Model Key:  DeepLearning_grid_0_AutoML_20180430_190847_model_15


ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.000352499900281
RMSE: 0.0187749806999
LogLoss: 0.00630721151319
Mean Per-Class Error: 0.0
AUC: 1.0
Gini: 1.0
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.799799249545: 


,-1,1,Error,Rate
-1,439.0,0.0,0.0,(0.0/439.0)
1,0.0,388.0,0.0,(0.0/388.0)
Total,439.0,388.0,0.0,(0.0/827.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.7997992,1.0,197.0
max f2,0.7997992,1.0,197.0
max f0point5,0.7997992,1.0,197.0
max accuracy,0.7997992,1.0,197.0
max precision,0.9999986,1.0,0.0
max recall,0.7997992,1.0,197.0
max specificity,0.9999986,1.0,0.0
max absolute_mcc,0.7997992,1.0,197.0
max min_per_class_accuracy,0.7997992,1.0,197.0
max mean_per_class_accuracy,0.7997992,1.0,197.0


Gains/Lift Table: Avg response rate: 46.92 %



...

Scoring History: 


,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_logloss,training_auc,training_lift,training_classification_error,validation_rmse,validation_logloss,validation_auc,validation_lift,validation_classification_error
,2018-04-30 19:19:07,0.000 sec,None,0.0,0,0.0,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
,2018-04-30 19:19:07,9 min 33.216 sec,80291 obs/sec,10.0,1,8270.0,0.4892624,0.7029777,0.7002178,1.6577892,0.3639661,0.6690612,3.7679843,0.4904174,1.3809524,0.5172414
,2018-04-30 19:19:12,9 min 38.319 sec,272834 obs/sec,1670.0,167,1381090.0,0.0187750,0.0063072,1.0,2.1314433,0.0,0.6841549,11.1996938,0.4904174,0.8474026,0.5172414


Variable Importances: 


variable,relative_importance,scaled_importance,percentage
PC6,1.0,1.0,0.0187212
PC21,0.9952371,0.9952371,0.0186321
PC27,0.9743353,0.9743353,0.0182408
PC23,0.9669386,0.9669386,0.0181023
PC40,0.9458159,0.9458159,0.0177068
---,---,---,---
PC78,0.1785424,0.1785424,0.0033425
PC69,0.1769234,0.1769234,0.0033122
PC65,0.1668318,0.1668318,0.0031233
PC68,0.1650396,0.1650396,0.0030897



See the whole table with table.as_data_frame()
